# Building an Anime Recommender System

In [1]:
import sys
sys.path.append("..")

from animerec import models
from animerec import utils

import pandas
import pickle

### 1. Load Dataset and Model

In [2]:
# Load anime metadata.
pickle_in = open("../data/animes.pickle", 'rb')
animes = pickle.load(pickle_in)
animes.index = animes['anime_id']
animes = animes[ ['title', 'title_english', 'type', 'premiered', 'genre'] ]
pickle_in.close()

# Load model trained on server.
pickle_in = open("../model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

# Load anime id's.
pickle_in = open("../model/item-id.pickle", 'rb')
item_id = pickle.load(pickle_in)
pickle_in.close()

### 2. Webscrape current profile and create prediction

In [3]:
myprofile = utils.get_user_anime_list('Digib')
x = utils.get_score_vector_from_user_anime_list(myprofile, item_id)

Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=0&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=300&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=600&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=900&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=1200&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=1500&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=1800&status=7
Scraped 300 additional anime from https://myanimelist.net/animelist/Digibro/load.json?offset=2100&status=7


In [4]:
rhat = model.predict(x)

In [5]:
prediction = utils.prediction_to_dataframe(rhat, myprofile, item_id, keep_all=False)
prediction = pandas.DataFrame(prediction)
prediction.columns = ['score']
prediction = prediction.join(animes)

In [6]:
prediction.sort_values(by='score', ascending=False).head(20)

,score,title,title_english,type,premiered,genre
2476,11.161383,School Days,School Days,TV,Summer 2007,"Harem, Drama, Romance, School"
21,10.509492,One Piece,One Piece,TV,Fall 1999,"Action, Adventure, Comedy, Super Power, Drama,..."
268,10.401553,Golden Boy,Golden Boy,OVA,NaN,"Adventure, Comedy, Ecchi"
8861,10.367249,"Yosuga no Sora: In Solitude, Where We Are Leas...",Yosuga no Sora,TV,Fall 2010,"Drama, Ecchi, Harem, Romance"
6030,10.082407,Needless,Needless,TV,Summer 2009,"Action, Comedy, Ecchi, Sci-Fi, Super Power"
1535,10.037758,Death Note,Death Note,TV,Fall 2006,"Mystery, Police, Psychological, Supernatural, ..."
223,9.973504,Dragon Ball,Dragon Ball,TV,Winter 1986,"Adventure, Comedy, Fantasy, Martial Arts, Shou..."
245,9.930509,Great Teacher Onizuka,Great Teacher Onizuka,TV,Summer 1999,"Comedy, Drama, School, Shounen, Slice of Life"
4382,9.844774,Suzumiya Haruhi no Yuuutsu (2009),The Melancholy of Haruhi Suzumiya Season 2,TV,Spring 2009,"Sci-Fi, Slice of Life, Mystery, Comedy, Parody..."
66,9.822395,Azumanga Daioh,Azumanga Daioh: The Animation,TV,Spring 2002,"Slice of Life, Comedy, School"


In [7]:
filt = prediction.apply(lambda x: "Sci-Fi" in str(x.genre), axis=1)
prediction[filt].sort_values(by='score', ascending=False).head(60)

,score,title,title_english,type,premiered,genre
6030,10.082407,Needless,Needless,TV,Summer 2009,"Action, Comedy, Ecchi, Sci-Fi, Super Power"
4382,9.844774,Suzumiya Haruhi no Yuuutsu (2009),The Melancholy of Haruhi Suzumiya Season 2,TV,Spring 2009,"Sci-Fi, Slice of Life, Mystery, Comedy, Parody..."
918,9.822210,Gintama,Gintama,TV,Spring 2006,"Action, Sci-Fi, Comedy, Historical, Parody, Sa..."
379,9.797571,Heppoko Jikken Animation Excel♥Saga,Excel Saga,TV,Fall 1999,"Comedy, Parody, Sci-Fi, Shounen"
9969,9.631775,Gintama&#039;,Gintama Season 2,TV,Spring 2011,"Action, Sci-Fi, Comedy, Historical, Parody, Sa..."
820,9.502004,Ginga Eiyuu Densetsu,Legend of the Galactic Heroes,OVA,NaN,"Military, Sci-Fi, Space, Drama"
28977,9.465961,Gintama°,Gintama Season 4,TV,Spring 2015,"Action, Comedy, Historical, Parody, Samurai, S..."
95,9.257932,Turn A Gundam,∀ Gundam,TV,Spring 1999,"Action, Military, Sci-Fi, Adventure, Space, Dr..."
967,8.937898,Hokuto no Ken,Fist of the North Star,TV,Fall 1984,"Action, Sci-Fi, Drama, Martial Arts, Shounen"
15417,8.567271,Gintama&#039;: Enchousen,Gintama: Enchousen,TV,Fall 2012,"Action, Comedy, Historical, Parody, Samurai, S..."


In [8]:
filt = prediction.apply(lambda x: "Fantasy" in str(x.genre), axis=1)
prediction[filt].sort_values(by='score', ascending=False).head(60)

,score,title,title_english,type,premiered,genre
21,10.509492,One Piece,One Piece,TV,Fall 1999,"Action, Adventure, Comedy, Super Power, Drama,..."
223,9.973504,Dragon Ball,Dragon Ball,TV,Winter 1986,"Adventure, Comedy, Fantasy, Martial Arts, Shou..."
3588,9.737680,Soul Eater,Soul Eater,TV,Spring 2008,"Action, Fantasy, Comedy, Supernatural, Shounen"
30694,9.105540,Dragon Ball Super,Dragon Ball Super,TV,Summer 2015,"Action, Adventure, Comedy, Fantasy, Martial Ar..."
4719,9.083586,Queen&#039;s Blade: Rurou no Senshi,Queen&#039;s Blade: The Exiled Virgin,TV,Spring 2009,"Action, Adventure, Ecchi, Fantasy"
6033,9.045630,Dragon Ball Kai,Dragon Ball Z Kai,TV,Spring 2009,"Action, Adventure, Comedy, Fantasy, Martial Ar..."
33,8.777414,Kenpuu Denki Berserk,Berserk,TV,Fall 1997,"Action, Adventure, Demons, Drama, Fantasy, Hor..."
427,8.642975,Kaleido Star,Kaleido Star,TV,Spring 2003,"Comedy, Sports, Drama, Fantasy, Shoujo"
14837,8.538204,Dragon Ball Z Movie 14: Kami to Kami,Dragon Ball Z: Battle of Gods,Movie,NaN,"Action, Adventure, Fantasy, Martial Arts, Shou..."
10033,8.467440,Toriko,Toriko,TV,Spring 2011,"Action, Adventure, Comedy, Fantasy, Shounen"


In [9]:
filt = prediction.apply(lambda x: "Slice of Life" in str(x.genre), axis=1)
prediction[filt].sort_values(by='score', ascending=False).head(60)

,score,title,title_english,type,premiered,genre
245,9.930509,Great Teacher Onizuka,Great Teacher Onizuka,TV,Summer 1999,"Comedy, Drama, School, Shounen, Slice of Life"
4382,9.844774,Suzumiya Haruhi no Yuuutsu (2009),The Melancholy of Haruhi Suzumiya Season 2,TV,Spring 2009,"Sci-Fi, Slice of Life, Mystery, Comedy, Parody..."
66,9.822395,Azumanga Daioh,Azumanga Daioh: The Animation,TV,Spring 2002,"Slice of Life, Comedy, School"
2921,9.236265,Ashita no Joe 2,Rocky Joe 2,TV,Fall 1980,"Action, Drama, Shounen, Slice of Life, Sports"
10165,9.099749,Nichijou,Nichijou - My Ordinary Life,TV,Spring 2011,"Slice of Life, Comedy, School"
8675,8.917202,Seitokai Yakuindomo,NaN,TV,Summer 2010,"Comedy, School, Shounen, Slice of Life"
1698,8.612188,Nodame Cantabile,Nodame Cantabile,TV,Winter 2007,"Music, Slice of Life, Comedy, Drama, Romance, ..."
11843,8.607602,Danshi Koukousei no Nichijou,Daily Lives of High School Boys,TV,Winter 2012,"Slice of Life, Comedy, School"
2402,8.544494,Ashita no Joe,Tomorrow&#039;s Joe,TV,Spring 1970,"Action, Drama, Shounen, Slice of Life, Sports"
322,8.484519,Paradise Kiss,Paradise Kiss,TV,Fall 2005,"Slice of Life, Comedy, Drama, Romance, Josei"
